# NYS Education Data Exploration
This notebook explores the raw data files and identifies key fields for our dashboard

In [6]:
import pandas as pd 
import subprocess
import numpy as np 
from pathlib import Path 
import warnings
warnings.filterwarnings('ignore')

In [8]:
# set up paths 
DATA_DIR = Path('../data/raw')
PROCESSED_DIR = Path('../data/processed')
PROCESSED_DIR.mkdir(exist_ok=True)
print("Starting data exploration...")

Starting data exploration...


In [9]:
# Helper function: extract access db tables using mdbtools
def list_mdb_tables(db_path):
    """list all tables in an access database using mdbtools"""
    try:
        result = subprocess.run(
            ['mdb-tables', '-1', str(db_path)],
            capture_output=True,
            text=True,
            check=True
        )
        tables = [t.strip() for t in result.stdout.strip().split('\n') if t.strip()]
        return tables 
    except FileNotFoundError:
        print("ERROR: mdbtools not found!")
        print("Install it with: brew install mdbtools")
        return []
    except subprocess.CalledProcessError as e:
        print(f"Error lising tables: {e}")
        return []

def export_mdb_table_to_csv(db_path, table_name, output_path):
    """Export a table from access database to csv using mdbtools """
    try:
        result = subprocess.run(
            ['mdb-export', str(db_path), table_name],
            capture_output=True,
            text=True,
            check=True
        )
        output_path.parent.mkdir(parents=True, exist_ok=True)
        with open(output_path, 'w') as f:
            f.write(result.stdout)

            print(f"Exported {table_name} to {output_path}")
            return True 
    except subprocess.CalledProcessError as e:
        print(f"Error exporting {table_name}: {e}")
        return False 

def read_mdb_table(db_path, table_name):
    """ Read a table from access database into pandas dataframe"""
    try:
        result = subprocess.run(
            ['mdb-export', str(db_path), table_name],
            capture_output=True,
            text=True,
            check=True 
        )

        #parse csv output into dataframe
        from io import StringIO 
        df = pd.read_csv(StringIO(result.stdout))
        return df 
    except Exception as e:
        print(f"Error reading: {table_name}: {e}")
        return None 

In [22]:
#read excel file
print("\n" + "="*60)
print("EXPLORING: ELA, Math, and Science Refusals")
print("="*60)

refusals_file = DATA_DIR /'2024 Districts, Charters Elementary and Middle Grades ELA, Math, and Science Refusals.xlsx'

try:
    excel_file = pd.ExcelFile(refusals_file)
    print(f"\nSheets in workbook: {excel_file.sheet_names}")

    #read each sheet (skip information)
    data_sheets = ['ELA', 'MATH', 'SCIENCE']
    for sheet_name in excel_file.sheet_names:
        print(f"\n{'='*60}")
        print(f"SHEET: {sheet_name}")
        print('='*60)
        df = pd.read_excel(refusals_file, sheet_name=sheet_name)
        
        print(f"Shape: {df.shape}")
        print(f"\nColumn names:")
        print(df.columns.tolist())

        print(f"\nFirst 5 rows:")
        print(df.head())

        #look for location columns
        location_cols = [col for col in df.columns if any(x in str(col).lower() for x in['district', 'county', 'entity', 'name', 'institution'])]
        print(f"\\nLocation-related columns: {location_cols}")

        #check for NYC, Westchester, Nassau, Suffolk
        if location_cols:
            sample_col = location_cols[0] if location_cols else df.colums[0]
            print(f"\nSample values from '{sample_col}':")
            print(df[sample_col].value_counts().head(10))
except Exception as e:
    print(f"Error reading Excel file: {e}")


EXPLORING: ELA, Math, and Science Refusals

Sheets in workbook: ['Information', 'ELA', 'MATH', 'SCIENCE']

SHEET: Information
Shape: (11, 2)

Column names:
['2024 Districts, Charters Grades 3-8 ELA, 3-8 Math, and 5 & 8 Science Refusals', 'Unnamed: 1']

First 5 rows:
  2024 Districts, Charters Grades 3-8 ELA, 3-8 Math, and 5 & 8 Science Refusals  \
0                                                NaN                              
1  This file contains Grades 3-8 ELA, 3-8 Mathema...                              
2                                                NaN                              
3                                                NaN                              
4                                                NaN                              

  Unnamed: 1  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  
\nLocation-related columns: ['2024 Districts, Charters Grades 3-8 ELA, 3-8 Math, and 5 & 8 Science Refusals', 'Unnamed: 1']

Sample values from

In [24]:
#read exported csv files from access databases
print("\n" + "="*60)
print("EXPLORING: Exported CSV files from Access databases")
print("="*60)

print("\nNote: Run 'python export_access_tables.py' first to export")
print("the Access database tables to CSV files.\n")

#list of all csv files in processed directory
csv_files = list(PROCESSED_DIR.glob("*.csv"))

if csv_files:
    print(f"Found {len(csv_files)} CSV files:\n")

    enroll_files = [f for f in csv_files if f.name.startswith('ENROLL_')]
    studed_files = [f for f in csv_files if f.name.startswith('STUDED_')]

    print("ENROLLMENT FILES:")
    for f in enroll_files:
        print(f" -{f.name}")

        #explore a few key files
        print("\n" + "="*60)
        print("SAMPLE DATA FROM KEY FILES")
        print("="*60)

        if enroll_files:
            enroll_file = enroll_files[0]
            print(f"\n--- {enroll_file.name} ---")
            df = pd.read_csv(enroll_file)
            print(f"Shape: {df.shape}")
            print(f"\nColumns: {df.columns.tolist()}")
            print(f"\nFirst few rows:")
            print(df.head())

            if 'ENTITY_NAME' in df.columns:
                print("\nSearching for target regions...")
                target_keywords = ['NYC', 'NEW YORK CITY', 'WESTCHESTER', 'NASSAU', 'SUFFOLK']
                for keyword in target_keywords:
                    matches = df[df['ENTITY_NAME'].str.contains(keyword, case=False, na=FALSE)]
                    if len(matches) > 0:
                        print(f"\n'{keyword}' matches: {len(matches)}")
                        print(matches[['ENTITY_CD', 'ENTITY_NAME']].head())
        if studed_files:
            studed_file = studed_files[0]
            print(f"\n--- {studed_file.name} ---")
            df = pd.read_csv(studed_file)
            print(f"Shape: {df.shape}")
            print(f"\nColumns: {df.columns.tolist()}")
            print(f"\nFirst few rows:")
            print(df.head())
else:
    print("No CSV files found. Run 'python export_access_tables.py' first!")


EXPLORING: Exported CSV files from Access databases

Note: Run 'python export_access_tables.py' first to export
the Access database tables to CSV files.

Found 12 CSV files:

ENROLLMENT FILES:
 -ENROLL_BOCES_and_N_RC.csv

SAMPLE DATA FROM KEY FILES

--- ENROLL_BOCES_and_N_RC.csv ---
Shape: (14240, 12)

Columns: ['INSTITUTION_ID', 'ENTITY_CD', 'SCHOOL_NAME', 'YEAR', 'DISTRICT_CD', 'DISTRICT_NAME', 'BOCES_CD', 'BOCES_NAME', 'COUNTY_CD', 'COUNTY_NAME', 'NEEDS_INDEX', 'NEEDS_INDEX_DESCRIPTION']

First few rows:
   INSTITUTION_ID    ENTITY_CD                     SCHOOL_NAME  YEAR  \
0    800000055730  10100010014        MONTESSORI MAGNET SCHOOL  2023   
1    800000055731  10100010016    PINE HILLS ELEMENTARY SCHOOL  2023   
2    800000055732  10100010018       DELAWARE COMMUNITY SCHOOL  2023   
3    800000055733  10100010019  NEW SCOTLAND ELEMENTARY SCHOOL  2023   
4    800000055736  10100010023     ALBANY SCHOOL OF HUMANITIES  2023   

   DISTRICT_CD   DISTRICT_NAME  BOCES_CD  \
0    1010

In [25]:
#next steps
print("\n" + "="*60)
print("NEXT STEPS:")
print("="*60)
print("""
1. Review the table names and columns above
2. Identify which tables contain the data we need
3. Filter for NYC, Westchester, Nassau, and Suffolk counties
4. Create cleaned, merged dataset
After running this, share the output and we'll build the 
data processing pipeline to create our dashboard dataset.
 """)


NEXT STEPS:

1. Review the table names and columns above
2. Identify which tables contain the data we need
3. Filter for NYC, Westchester, Nassau, and Suffolk counties
4. Create cleaned, merged dataset
After running this, share the output and we'll build the 
data processing pipeline to create our dashboard dataset.
 


In [ ]:
#export all tables to csv
"""
print("\n" + "="*60)
print("EXPORTING ALL TABLES TO CSV")
print("="*60)

for db_name, db_file in [('ENROLL', enroll_file), ('STUDED', studed_file)]:
    if db_file.exists():
        tables = list_mdb_tables(db_file)
        for table in tables:
            output_file = PROCESSED_DIR /f"{db_name}_{table}.csv" 
            export_mdb_table_to_csv(db_file, table, output_file)
        print(f"Exported all {db_name} tables to {PROCESSED_DIR}")
"""